In [2]:
import pandas as pd
import numpy as np
import requests

In [3]:
def fetch_canonical_transcript_info(ensembl_gene_id):
    server = "https://rest.ensembl.org"
    ext = f"/lookup/id/{ensembl_gene_id}?expand=1"
    headers = {"Content-Type": "application/json"}

    # Fetch gene information
    response = requests.get(server + ext, headers=headers)
    if not response.ok:
        response.raise_for_status()

    gene_data = response.json()
    transcripts = gene_data.get("Transcript", [])

    # Find the canonical transcript
    canonical_transcript_info = None
    for transcript in transcripts:
        if transcript.get("is_canonical"):
            canonical_transcript_info = {
                "transcript_id": transcript["id"],
                "display_name": transcript.get("display_name", "Unknown Protein")
            }
            break

    return canonical_transcript_info

def fetch_protein_sequence(ensembl_transcript_id):
    server = "https://rest.ensembl.org"
    ext = f"/sequence/id/{ensembl_transcript_id}?type=protein"
    headers = {"Content-Type": "application/json"}

    response = requests.get(server + ext, headers=headers)
    if not response.ok:
        response.raise_for_status()

    protein_data = response.json()
    return protein_data.get("seq", "")

# Example usage
ensembl_gene_id = "ENSDARG00000045544"  # Replace with your Ensembl gene ID

# Step 1: Fetch canonical transcript information from Ensembl
canonical_transcript_info = fetch_canonical_transcript_info(ensembl_gene_id)

if canonical_transcript_info:
    transcript_id = canonical_transcript_info["transcript_id"]
    display_name = canonical_transcript_info["display_name"]
    
    # Step 2: Fetch protein sequence for the canonical transcript
    protein_sequence = fetch_protein_sequence(transcript_id)
    
    if protein_sequence:
        print(f"Query Ensembl Gene ID: {ensembl_gene_id}")
        print(f"Canonical Transcript Ensembl ID: {transcript_id}")
        print(f"Name: {display_name}")
        print(f"Protein Sequence:\n{protein_sequence}")
    else:
        print(f"Protein sequence not found for the canonical transcript ID {transcript_id}.")
else:
    print(f"No canonical transcript found for the given Ensembl gene ID {ensembl_gene_id}.")



Query Ensembl Gene ID: ENSDARG00000045544
Canonical Transcript Ensembl ID: ENSDART00000100952
Name: hgfa-202
Protein Sequence:
MRSDIMWMYQALLFVVLTVNVDCRKQTLQRYQKSENSRLLCTDCPESPRIRNLSLEECARKCSKSKKSCRAFYFDHINRKCHFLPFDRFSEGAKRERKPSCDLYEKKDYVRECIIGSGVNYKGRRSFTKTGITCQSWNMSVPHEHNFKPTRHKKSDLRQNFCRNPDNDPNGPWCFTELTETRHQDCGLPQCSDVECMKCNGETYRGPMDHTESGKECQRWDSQKPHKHTYQPHRHVGKGLDDNFCRNPNNDVRPWCYTMDKNTPWEYCNISVCDSDSDVEVEVTSSCFRGQGEGYRGTVNVTPAGVTCQRWDALSPHIHSYTPHNYKCKDLRENYCRNPDGSEIPWCFTTDAKVRKAFCTNIPRCESESSDSTECYEDNGESYRGNLSKTRSGIPCGLWSDHTFRRDTRSAKASAGLELNLCRNPDRDKHGPWCYTSNSSIPWDYCGLERCGGPKPSCFIHKTTRIVGGMRVQRAEDGSWVVSIQKGNRHWCGGSLIREEWVLTDQQCFSTCVPDLSEYTVQVGLLHLNASAGTQALRIAHVVCGPEGSNLALLKLTTPAPLSEHVRTVQLPVAGCAVAEGTLCLMYGWGDTKGHEGSLKMVGLPIVSNKRCSQSHNGILPITETKICAGGKRDQGVCEKDYGGPLVCQEGESKVIVGVSINGRGCAVARRPAVFVNVAFYSEWIRKVFKYYSDMEISY


### Zebra embedding

In [17]:
genes_id = ['ENSDARG00000002952', 'ENSDARG00000006939', 'ENSDARG00000007329', 'ENSDARG00000007406', 'ENSDARG00000008174', 'ENSDARG00000008305', 'ENSDARG00000015314', 'ENSDARG00000017121', 'ENSDARG00000021032', 'ENSDARG00000021201', 'ENSDARG00000031891', 'ENSDARG00000036292', 'ENSDARG00000042826', 'ENSDARG00000045544', 'ENSDARG00000054454', 'ENSDARG00000058822', 'ENSDARG00000059279', 'ENSDARG00000070546', 'ENSDARG00000070903', 'ENSDARG00000101576', 'ENSDARG00000104566', 'ENSDARG00000114554', 'ENSDARG00000078507']
genes_name = ['smo', 'tbx16l', 'tbx16', 'phox2a', 'hoxb1a', 'hand2', 'zc4h2', 'mafba', 'foxd3', 'noto', 'tbx1', 'cdx4', 'egr2b', 'hgfa', 'epha4a', 'wnt3a', 'tfap2a', 'msgn1', 'met', 'tbxta', 'foxi1', 'cdx1a', 'wnt8']

In [18]:
# ensembl_to_do = ["ENSDARG00000045544"]
ensembl_to_do = genes_id

counter = 0
with open("zebra_protein_sequences.fasta", "w") as fasta_file:
    for ensembl_gene_id in ensembl_to_do:
        canonical_transcript_info = fetch_canonical_transcript_info(ensembl_gene_id)
        if canonical_transcript_info:
            counter += 1
            transcript_id = canonical_transcript_info["transcript_id"]
            display_name = canonical_transcript_info["display_name"]
            protein_sequence = fetch_protein_sequence(transcript_id)
            if protein_sequence:
                fasta_file.write(f">query_{ensembl_gene_id}_canonical_transcript_{transcript_id}_Name_{display_name}\n")
                fasta_file.write(f"{protein_sequence}\n")
            else:
                print(f"Protein sequence not found for the canonical")
print(f"found {counter} canonical transcripts, from {len(ensembl_to_do)} genes.")

found 23 canonical transcripts, from 23 genes.


### Norman gene embeddings

In [12]:
fname = "/network/scratch/g/guillaume.huguet/projects/esm/norman_genes.csv"
df = pd.read_csv(fname)
ensembl_gene_ids = df["index"].values
ensembl_to_do = list(ensembl_gene_ids)

In [13]:
counter = 0
with open("norman_protein_sequences.fasta", "w") as fasta_file:
    for ensembl_gene_id in ensembl_to_do:
        canonical_transcript_info = fetch_canonical_transcript_info(ensembl_gene_id)
        if canonical_transcript_info:
            counter += 1
            transcript_id = canonical_transcript_info["transcript_id"]
            display_name = canonical_transcript_info["display_name"]
            protein_sequence = fetch_protein_sequence(transcript_id)
            if protein_sequence:
                fasta_file.write(f">query_{ensembl_gene_id}_canonical_transcript_{transcript_id}_Name_{display_name}\n")
                fasta_file.write(f"{protein_sequence}\n")
            else:
                print(f"Protein sequence not found for the canonical")
print(f"found {counter} canonical transcripts, from {len(ensembl_to_do)} genes.")

found 105 canonical transcripts, from 105 genes.


# Satijas

In [22]:
fname = "/network/scratch/g/guillaume.huguet/projects/esm/perturb_emb/satijas_v2/staj_gene_ids.txt"
df = pd.read_csv(fname, sep="\t", header=None)
ensembl_to_do = df.values.tolist()
# flatten
ensembl_to_do = [item for sublist in ensembl_to_do for item in sublist]


In [27]:
counter = 0
with open("satijas_protein_sequences.fasta", "w") as fasta_file:
    for ensembl_gene_id in ensembl_to_do:
        canonical_transcript_info = fetch_canonical_transcript_info(ensembl_gene_id)
        if canonical_transcript_info:
            counter += 1
            print(counter)
            transcript_id = canonical_transcript_info["transcript_id"]
            display_name = canonical_transcript_info["display_name"]
            protein_sequence = fetch_protein_sequence(transcript_id)
            if protein_sequence:
                fasta_file.write(f">query_{ensembl_gene_id}_canonical_transcript_{transcript_id}_Name_{display_name}\n")
                fasta_file.write(f"{protein_sequence}\n")
            else:
                print(f"Protein sequence not found for the canonical")
print(f"found {counter} canonical transcripts, from {len(ensembl_to_do)} genes.")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
found 223 canonical transcripts, from 223 genes.
